# Computing readcounts from ENCODE .bam files

Requirements:
- Python packages: pandas, numpy, os, subprocess
- bedtools
- samtools

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
from functools import reduce

In [2]:
!pwd

/mnt/c/Users/ochapman/Documents/CSE 180 TA/Week 7/discussion


## MergePeaks
Takes the peaks in our different experiments and merges them into consensus peaks.

In [3]:
# INPUTS
# list of bed files I want to merge
# OUTPUTS
# narrowPeaks.merge.bed,
# narrowPeaks.merge.saf

files = ['10d_final.replicated.narrowPeak.bed','13d_final.replicated.narrowPeak.bed'] # list of wanted files
files = reduce(lambda a,b: a + " " + b,files) # as a single string
files = '"'+files+'"' # add quotes
script = "./mergePeaks.sh"
command = '{} {} {}'.format(script, '.', files)
print(command) # if you want to execute it outside this notebook in a console
#subprocess.check_call(command, shell=True)
p = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
stdout, stderr = p.communicate()
print("Stdout:\n",stdout.decode('UTF-8'))
print("Stderr:\n",stderr.decode('UTF-8'))

./mergePeaks.sh . "10d_final.replicated.narrowPeak.bed 13d_final.replicated.narrowPeak.bed"
Stdout:
 Loading peaks...
Sorting peaks by chromosome...
Merging peaks...
Removing intermediate files...
Converting bed to SAF file for featureCounts annotation...
Done.

Stderr:
 ./mergePeaks.sh: line 20: module: command not found



## CountReads
Counts the number of reads overlapping each consensus peak (from MergePeaks) for each sample.

Strongly suggest you do this on the cluster, as bam files are not small.

In [4]:
# FeatureCounts version
# DEPENDENCIES
# MergePeaks .saf file
# .bam files
# OUTPUTS
# narrow_peak_counts.raw.txt

# Do CountReads locally, which again, I cannot recommend
saf = "narrowPeaks.merge.saf"
out = "narrow_peak_counts.raw.txt"
files = '10d_r1.bam','10d_r2.bam','13d_r1.bam','13d_r2.bam'
files = reduce(lambda a,b: a + " " + b,files) # as a single string
command = 'featureCounts -a {} -o {} {} -F SAF -T 4'.format(saf,out,files)
print(command)

featureCounts -a narrowPeaks.merge.saf -o narrow_peak_counts.raw.txt 10d_r1.bam 10d_r2.bam 13d_r1.bam 13d_r2.bam -F SAF -T 4


In [5]:
# Alternate Bedtools version
# DEPENDENCIES
# MergePeaks .bed file
# .bam files
# OUTPUTS
# narrow_peak_counts.raw.txt
merge = 'narrowPeaks.merge.bed'
files = [
    '10d_r1',
    '10d_r2',
    '13d_r1',
    '13d_r2'
]
for f in files:
    print('bedtools coverage -counts -sorted -a {} -b {}.bam > {}.coverage.bed'.format(merge,f,f))
# run these in command line

bedtools coverage -counts -sorted -a narrowPeaks.merge.bed -b 10d_r1.bam > 10d_r1.coverage.bed
bedtools coverage -counts -sorted -a narrowPeaks.merge.bed -b 10d_r2.bam > 10d_r2.coverage.bed
bedtools coverage -counts -sorted -a narrowPeaks.merge.bed -b 13d_r1.bam > 13d_r1.coverage.bed
bedtools coverage -counts -sorted -a narrowPeaks.merge.bed -b 13d_r2.bam > 13d_r2.coverage.bed


## Convert to FPKM
(TODO)

## Generate .gct file

In [6]:
# Create a pandas dataframe
files = [
    '10d_r1.coverage.bed',
    '10d_r2.coverage.bed',
    '13d_r1.coverage.bed',
    '13d_r2.coverage.bed'
]
flag = False
for f in files:
    name = f.split('.')[0]
    if not flag:
        counts = pd.read_csv(f,sep='\t',header=None,names=['chr','start','stop',name])
        flag = True
    else:
        temp = pd.read_csv(f,sep='\t',header=None,names=['chr','start','stop',name])
        counts[name]=temp[name]
counts

,chr,start,stop,10d_r1,10d_r2,13d_r1,13d_r2
0,chr1,3044733,3044894,13,8,2,3
1,chr1,3045431,3045627,10,13,5,1
2,chr1,3147748,3147913,14,11,6,4
3,chr1,3234967,3235105,10,11,9,3
4,chr1,3247463,3247651,4,11,8,7
5,chr1,3335675,3335856,10,15,6,2
6,chr1,3621816,3622359,23,29,16,17
7,chr1,3653638,3653934,19,10,7,7
8,chr1,3654164,3654434,8,13,11,13
9,chr1,3655699,3656377,27,30,22,27


In [7]:
# write a gct

# format the index
counts['NAME'] = counts.chr.map(str)+':'+counts.start.map(str)+'-'+counts.stop.map(str)
counts.set_index('NAME', inplace=True)
counts.drop(['chr','start','stop'],inplace=True,axis='columns')
counts['Description']='NA'
cols = list(counts.columns)
cols = [cols[-1]]+cols[:-1]
counts = counts[cols]

# write gct header
file = 'rawcounts.gct'
with open(file,'w') as f:
    f.write('#1.2\n')
    f.write('{}\t{}\n'.format(counts.shape[0],counts.shape[1]))
counts.to_csv(file,sep='\t',mode='a')
counts

,Description,10d_r1,10d_r2,13d_r1,13d_r2
NAME,,,,,
chr1:3044733-3044894,NA,13,8,2,3
chr1:3045431-3045627,NA,10,13,5,1
chr1:3147748-3147913,NA,14,11,6,4
chr1:3234967-3235105,NA,10,11,9,3
chr1:3247463-3247651,NA,4,11,8,7
chr1:3335675-3335856,NA,10,15,6,2
chr1:3621816-3622359,NA,23,29,16,17
chr1:3653638-3653934,NA,19,10,7,7
chr1:3654164-3654434,NA,8,13,11,13
